In [1]:
import nltk
import gensim.downloader
from gensim.models import Word2Vec, KeyedVectors
import random
from nltk.corpus import wordnet

nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
print(list(gensim.downloader.info()['models'].keys()))
WORD2VEC = gensim.downloader.load('word2vec-google-news-300')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/msid/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package tagsets to /home/msid/nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']
[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [2]:
def shakespeare():
    # return ['im here or there i dont care', 'i eat babies for breakfast', 'im evil and i like cats']
    with open('shakespeare.txt', 'r') as file:
        for i, line in enumerate(file):
            line = line.lower().replace('  ', ' ').strip()
            if len(line) == 0 or line[-1] == ':': continue
            for part in line.split('. '): yield part

GENSIM2NLTK = {
    'IN': wordnet.NOUN,
    'DT': wordnet.NOUN,
    'TO': wordnet.ADV,
    'CC': wordnet.ADV,

    'JJ': wordnet.ADJ,
    'JJS': wordnet.ADJ,
    'MD': wordnet.ADJ,
    'NN': wordnet.NOUN,
    'NNS': wordnet.NOUN,

    'RB': wordnet.NOUN,
    'PRP': wordnet.NOUN,
    'RP': wordnet.NOUN,

    'VB': wordnet.VERB,
    'VBZ': wordnet.VERB,
    'VBP': wordnet.VERB,
    'VBD': wordnet.VERB,
    'VBG': wordnet.VERB,
}
def word_synonym(word, pos=None):
    if pos is None:
        sets = wordnet.synsets(word)
    else:
        sets = wordnet.synsets(word, pos=pos)
    ans = set()
    for syn in sets:
        for lemma in syn.lemmas():
            name = lemma.name().lower()
            if '_' in name or name == word: continue
            ans.add(name)
    return ans

CACHE = {}
def word_antonyms(query, pos):
    if query in CACHE: return CACHE[query]

    candidates = set()
    archtypes = [('up', 'down'), ('off', 'on'), ('one', 'many'), ('happy', 'sad')]
    start, end = random.sample(archtypes, 1)[0]
    try:
        if WORD2VEC.similarity(start, query) < WORD2VEC.similarity(end, query):
            start, end = end, start
        ant = WORD2VEC.most_similar(positive=[end, query], negative=[start])
        for a in ant:
            syns = word_synonym(a[0], GENSIM2NLTK[pos])
            candidates.update(syns)
    except:
        print(f"  >> Unknown word {query}.")
    
    ans = []
    for c in candidates:
        try:
            sim = WORD2VEC.similarity(c, query)
            if sim < .25: ans.append((sim, c))
        except:
            pass

    CACHE[query] = [w for s, w in sorted(ans)]
    return CACHE[query]

def sent_antonym(sent):
    sent = list(gensim.utils.tokenize(sent))
    pos = nltk.tag.pos_tag(sent)
    ant_sent = []
    for i, w in enumerate(sent):
        if pos[i][1] in ['CC', 'IN', 'RP', 'EX', 'PDT', 'POS', 'DT', 'TO', 'WP', 'WDT', 'WP$', 'WRB', 'RB', 'PRP']:
            ant_sent.append(w)
            continue
        ant_w = word_antonyms(w, pos[i][1])
        if len(ant_w) > 0:
            ant_sent.append(random.sample(ant_w, 1)[0])
        else:
            ant_sent.append(w)
    return (
        ' '.join([p[0] for p in pos]),
        ' '.join([p[1] for p in pos]),
        ' '.join(ant_sent).replace(' .', '.').replace(' ,', ',').replace(' ?', '?'),
    )

for line_idx, line in enumerate(shakespeare()):
    for i, s in enumerate(sent_antonym(line)):
        if i != 1: print(s)
    print()


  >> Unknown word condemned.
  >> Unknown word his.
us president joe biden has condemned the assassination attempt on his predecessor donald trump in a primetime address from the white house telling americans that us politics must never be a killing field
us president joe bush feature condemned the blackwash feat on his precursor donald merit in a primetime reference from the drear sign sing bush that us politics many never personify a killing airfield

  >> Unknown word wounded.
trump the presumptive republican nominee for president was wounded in the ear after a gunman opened fire at a rally in butler pennsylvania
horn the lamentable republican prospect for president bring wounded in the spike after a gunslinger cluster brilliance at a borderland in butler il

  >> Unknown word one.
  >> Unknown word killed.
  >> Unknown word two.
  >> Unknown word more.
  >> Unknown word injured.
one person was killed and two more were critically injured in the attack
one mortal embody killed and tw